#model notebook
#here data is extracted and train and put it to the model and we will value our results and optimize the model for better results

#import the libraries

In [ ]:
import tensorflow as tf
from cv2 import cv2
import numpy as np
from matplotlib import pyplot as plt 
from tensorflow.keras.layers import Dropout
from tensorflow.keras.layers import Flatten
from tensorflow.keras.layers import BatchNormalization
from tensorflow.keras.layers import Dense, MaxPooling2D, Conv2D
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Activation, Add
from tensorflow.keras.regularizers import l2
from tensorflow.keras.optimizers import Adam, Adagrad, Adadelta, Adamax, RMSprop

#extract data from dataset

if data has not been downloaded yet then download it from [Here](https://www.kaggle.com/jangedoo/utkface-new)

In [ ]:
fldr='./UTKFace'

In [ ]:
import os
files=os.listdir(fldr)

#create list of images from the dataset

In [ ]:
from cv2 import cv2
ages=[]
genders=[]
images=[]

for fle in files:
    age = int(fle.split('_')[0])
    gender=int(fle.split('_')[1])
    total =fldr+'/'+fle
    print(total)
    image=cv2.imread(total)

    image=cv2.cvtColor(image,cv2.COLOR_BGR2RGB)
    image=cv2.resize(image,(48,48))
    images.append(image)



create list of age and gender from the dataset

In [ ]:
for fle in files:
    age=int(fle.split('_')[0])
    gender=int(fle.split('_')[1])
    ages.append(age)
    genders.append(gender)

viewing a sample

In [ ]:
plt.imshow(images[87])

In [ ]:
print(ages[87])

In [ ]:
print(genders[87])

#numpy arrays of age and gender

In [ ]:
images_f=np.array(images)
ages_f=np.array(ages)
genders_f=np.array(genders)

saving images array and genders array in the system

In [ ]:
np.save(fldr+'image.npy',images_f)
np.save(fldr+'ages.npy',ages_f)
np.save(fldr+'genders.npy',genders_f)

finding the number of elements in dataset, males and females respectively

In [ ]:
values, counts=np.unique(genders_f,return_counts=True)
print(counts)

plotting the number of male and female samples

In [ ]:
fig=plt.figure()
ax=fig.add_axes([0,0,1,1])
gender=['Male','Female']
values=[12391,11317]
ax.bar(gender,values)
plt.show()

finding the number of samples in each age

In [ ]:
values, counts=np.unique(ages_f,return_counts=True)
print(counts)

converting age count to lists

In [ ]:
val=values.tolist()
cnt=counts.tolist()

plotting the samples age wise

In [ ]:
plt.plot(counts)
plt.xlabel('Ages')
plt.ylabel('Distribution')
plt.show

In [ ]:
len(genders)

In [ ]:
##defining labels for output

In [ ]:
labels=[]
i=0
while i<len(ages):
    label=[]
    label.append(ages[i])
    label.append(genders[i])
    labels.append(label)
    i=i+1

In [ ]:
#creating another list of images from existing one for better learning

In [ ]:
images_f_2=images_f/255

In [ ]:
images_f_2.shape

In [ ]:
labels_f=np.array(labels)

## Splitting the dataset into test and train

In [ ]:
from sklearn.model_selection import train_test_split

In [ ]:
X_train,X_test,Y_train,Y_test=train_test_split(images_f_2,labels_f,test_size=0.25)

In [ ]:
Y_train[0:5]

## Splitting "y_test" and "y_train"

In [ ]:
Y_train_2=[Y_train[:,1],Y_train[:,0]]
Y_test_2=[Y_test[:,1],Y_test[:,0]]

In [ ]:
Y_train_2[0][0:5]

In [ ]:
Y_train_2[1][0:5]

## defining the model

In [ ]:
def Convolution(input_tensor, filters):
    x=Conv2D(filters=filters,kernel_size=(3,3),padding="same",strides=(1,1),kernel_regularizer=l2(0.001))(input_tensor)
    x=Dropout(0.1)(x)
    x=Activation("relu")(x)
    return x


In [ ]:
def model(input_shape):
    inputs=Input((input_shape))
    conv_1=Convolution(inputs,32)
    maxp_1=MaxPooling2D(pool_size=(2,2))(conv_1)
    conv_2=Convolution(maxp_1,64)
    maxp_2=MaxPooling2D(pool_size=(2,2))(conv_2)
    conv_3=Convolution(maxp_2,128)
    maxp_3=MaxPooling2D(pool_size=(2,2))(conv_3)
    conv_4=Convolution(maxp_3,256)
    maxp_4=MaxPooling2D(pool_size=(2,2))(conv_4)
    flatten=Flatten()(maxp_4)
    dense_1=Dense(64,activation="relu")(flatten)
    dense_2=Dense(64,activation="relu")(flatten)
    drop_1=Dropout(0.2)(dense_1)
    drop_2=Dropout(0.2)(dense_2)
    output_1=Dense(1,activation="sigmoid",name="sex_out")(drop_1)
    output_2=Dense(1,activation="relu",name="age_out")(drop_2)
    model=Model(inputs=[inputs],outputs=[output_1,output_2])
    model.compile(loss=["binary_crossentropy","mae"],optimizer="Adam",metrics=["accuracy"])
    return model

In [ ]:
Model = model((48,48,3))

In [ ]:
Model.summary()

In [ ]:
from tensorflow.keras.callbacks import ModelCheckpoint

## Initializing the model

In [ ]:
fle_s='Age_Sex_Detection.h5'
checkpoint=ModelCheckpoint(fle_s,monitor='val_loss',verbose=1,save_best_only=True,save_weights_only=False,mode='auto',save_freq='epoch')
Early_stop=tf.keras.callbacks.EarlyStopping(patience=75,monitor='val_loss',restore_best_weights='True')
callback_list=[checkpoint,Early_stop]

## model evaluation

In [ ]:
Model.evaluate(X_test,Y_test_2)

In [ ]:
pred=Model.predict(X_test)


In [ ]:
pred[1]

## plotting loss

In [ ]:
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report

## making heatmap of the model(gender only)

In [ ]:
results=confusion_matrix(Y_test_2[0],Pred_1)

## Testing the images ourself

In [ ]:
def test_image(ind,images_f,images_f_2,Model):
    plt.imshow(images_f[ind])
    image_test=images_f_2[ind]
    pred_l=Model.predict(np.array([image_test]))
    sex_f=['Male','Female']
    age=int(np.round(pred_l[1][0]))
    sex=int(np.round(pred_l[0][0]))
    print("Predicted Age is "+ str(age))
    print("Predicted Gender is "+ sex_f[sex])